In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Download a file based on its file ID.
download = drive.CreateFile({'id': '16hA5xU-EmCSRAVEkK_xtYQv8FWyvHOl6'})
download.GetContentFile('dataset.zip')

In [0]:
!unzip dataset.zip
!rm dataset.zip

In [0]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

# Initialising the CNN or classifier
classifier = Sequential()

# Step 1 - Convolution, pooling!
classifier.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 1)))
classifier.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(BatchNormalization())

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 3, activation = 'sigmoid'))

# Compiling the CNN or classifier
classifier.compile(optimizer = Adam(lr=3e-06, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True), loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

def plotHistory(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()    

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training', target_size = (224, 224),color_mode='grayscale',batch_size = 32,class_mode = 'categorical')

validation_set = test_datagen.flow_from_directory('dataset/validation', target_size = (224, 224), color_mode='grayscale', batch_size = 32, class_mode = 'categorical')

try:
  history = classifier.fit_generator(training_set, steps_per_epoch = 94, callbacks = [EarlyStopping('val_acc', mode='auto', patience=3)], epochs = 50, validation_data = validation_set, validation_steps = 37)
  plotHistory(history)
finally:
  classifier.save('model.h5')

In [0]:
# Library imports
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

def rgb2gray(gimage):
    gimage = 0.2989 * gimage[:,:,0] + 0.5870 * gimage[:,:,1] + 0.1140 * gimage[:,:,2]
    gimage = np.expand_dims(gimage, axis=2)
    return gimage
  
correct = 0
total_img = 0        
tumor_types = ['glioma', 'meningioma', 'pituitary']
model = load_model('model.h5')

# Detection accuracy
for index, tumor in enumerate(tumor_types):
        
    for i in range (1001, 1201):
        name = 'dataset/validation/' + tumor + '/' + tumor + str(i) + '.png'
        test_image = image.load_img(name, target_size = (224, 224))
        test_image = image.img_to_array(test_image)
        test_image = rgb2gray(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        result = model.predict(test_image)
        if result[0][index] == 1:
            correct = correct + 1
        total_img = total_img + 1

# Print accuracy
accuracy = float(correct) * 100 / float(total_img)
print(str(correct) + " out of " + str(total_img) + " are correctly classified")
print("acurracy of the model on this class is " + str(accuracy) + "%")

In [0]:
# upload weights file to google drive.
upload = drive.CreateFile({'title': 'model.h5'})
upload.SetContentFile('model.h5')
upload.Upload()
print('Uploaded file with ID {}'.format(upload.get('id')))